In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 15})
#%config InlineBackend.figure_format='retina'

# Riemann Sum

We were already introduced to a [Riemann sum](https://en.wikipedia.org/wiki/Riemann_sum)
which is a certain kind of approximation of an integral by a finite sum

$$
\int_a^b f(x) dx
\approx
\sum_{i = 1}^{N} f(x_i^*) \Delta x_i
\equiv S
$$

where $\Delta x_i = x_i - x_{i - 1}$, $x_i^* \in [x_{i - 1}, x_i]$, and

$$a = x_0 < x_1 < x_2 < \cdots < x_{n - 1} < x_n = b$$

![Partition](assets/partition.svg)

Some specific types of Riemann sums are:

- If $x_i^* = x_{i - 1}$, then $S$ is called a **left rule** or **left Riemann sum**.
- If $x_i^* = x_i$, then $S$ is called a **right rule** or **right Riemann sum**.
- If $x_i^* = (x_i + x_{i - 1})/2$, then $S$ is called a **midpoint rule** or **middle Riemann sum**.

We can think of a Riemann sum as the area of $N$ rectangles with heights determined by the graph of $y = f(x)$.
Let's visualize rectangles in the left, right and midpoint Riemann sums for the function

$$
f(x) = \frac{1}{1 + x^2}
$$

over the interval $[0, 5]$ with a partition of size $N = 10$.

In [ ]:
def f(x):
    return 1 / (1 + x*x)

def coord(x):
    return x, f(x)

a, b, N = 0, 5, 10
curve = coord(np.linspace(a, b, 200))

# partition
partition = np.linspace(a, b, N + 1)
left_x = partition[:-1]
right_x = partition[1:]
middle_x = (left_x + right_x) / 2
delta_x = right_x - left_x

# Riemann sums
left_sum = sum(f(left_x) * delta_x)
right_sum = sum(f(right_x) * delta_x)
middle_sum = sum(f(middle_x) * delta_x)

# drawing
plt.figure(figsize=(11, 4))

plt.subplot(1, 3, 1)
plt.plot(*curve, 'b')
plt.plot(*coord(left_x), 'bo')
plt.bar(*coord(left_x), label=f"sum = {left_sum:.6f}",
        align='edge', width=delta_x[0],
        alpha=0.2, edgecolor='b')
plt.title(f'Left Riemann Sum, N = {N}', fontsize=13)
plt.xlabel("$x$")
plt.ylabel("$f(x)$")
plt.legend(fontsize=13)

plt.subplot(1, 3, 2)
plt.plot(*curve, 'b')
plt.plot(*coord(middle_x), 'bo')
plt.bar(*coord(middle_x), label=f"sum = {middle_sum:.6f}",
        width=delta_x[0], alpha=0.2, edgecolor='b')
plt.title(f'Middle Riemann Sum, N = {N}', fontsize=13)
plt.xlabel("$x$")
plt.legend(fontsize=13)

plt.subplot(1, 3, 3)
plt.plot(*curve, 'b')
plt.plot(*coord(right_x), 'bo')
plt.bar(*coord(right_x), label=f"sum = {right_sum:.6f}",
        align='edge', width=-delta_x[0],
        alpha=0.2, edgecolor='b')
plt.title(f'Right Riemann Sum, N = {N}', fontsize=13)
plt.xlabel("$x$")
plt.legend(fontsize=13)

plt.tight_layout()
plt.show()

We know that the exact value is

$$
\int_0^5 \frac{1}{1 + x^2} dx = \arctan(5) \approx 1.37340076694501586086
$$

Comparing with the exact value, the middle Riemann sum is understandably most accurate.

## Error Analysis

Let's quantify the accuracy.
Without proof, the error formulas are given below.

----

Let $L_N(f)$ be the left Riemann sum

$$
L_N(f) = \sum_{i = 1}^N f(x_{i - 1}) \Delta x
$$

where $\Delta x = (b - a)/N$ and $x_i = a + i\Delta x$.
The error bound of $L_N(f)$ is

$$
E_N^L(f) = 
\left| \int_a^b f(x) dx - L_N(f) \right|
\le \frac{(b - a)^2}{2N} K_1
$$

where $K_1 = \max_{x \in (a, b)} |f'(x)|$.

----

Let $R_N(f)$ be the right Riemann sum

$$
R_N(f) = \sum_{i = 1}^N f(x_{i}) \Delta x
$$

where $\Delta x = (b - a)/N$ and $x_i = a + i\Delta x$.
The error bound of $R_N(f)$ is

$$
E_N^R(f) = 
\left| \int_a^b f(x) dx - R_N(f) \right|
\le \frac{(b - a)^2}{2N} K_1
$$

where $K_1 = \max_{x \in (a, b)} |f'(x)|$.

----

Let $M_N(f)$ be the middle Riemann sum

$$
M_N(f) = \sum_{i = 1}^N f(x_{i}^*) \Delta x
$$

where $\Delta x = (b - a)/N$ and $x_i^* = a + (i - 0.5)\Delta x$.
The error bound of $M_N(f)$ is

$$
E_N^M(f) = 
\left| \int_a^b f(x) dx - M_N(f) \right|
\le \frac{(b - a)^3}{24 N^2} K_2
$$

where $K_2 = \max_{x \in (a, b)} |f''(x)|$.

----

Several points worth highlighting:

- The left and right Riemann sums have the same error bound which depends on the first derivative $f'(x)$ and has order $O(N^{-1})$.
- The middle Riemann sum error bound depends on the second derivative $f''(x)$ and has order $O(N^{-2})$.
- Therefore, the middle Riemann sum converges faster.

## Implementation

In [ ]:
def riemann_sum(f, partition, method=0):
    """Compute the {left|middle|right} Reimann sum of f(x) given the partition.
    
    :param f:
        A function of one variable
    :param partition:
        A sequence of x partition points; the length must be greater than or equal to 2
    :param method:
        Either -1 (left sum), 0 (middle sum), or 1 (right sum)
        The default is 0.
    """
    
    assert method == 0 or method == -1 or method == 1, f'Unrecognized method={method}'
    
    dx = [r - l for (l, r) in zip(partition[:-1], partition[1:])]
    
    N  = len(partition) - 1
    li = 0 + int(method > 0)
    ri = 1 - int(method < 0)
    xi = [0.5 * (r + l) for (l, r) in
          zip(partition[li:(li+N)], 
              partition[ri:(ri+N)])
         ]
    
    return sum(f(x) * d for (x, d) in zip(xi, dx))

Let's test this to compute

$$
\int_0^{\pi/2} \sin x dx = 1
$$

In [ ]:
result = [
    riemann_sum(np.sin, np.linspace(0, np.pi/2, 1001), method=method)
    for method in (-1, 0, 1)
]

print("left_sum = {}, middle_sum = {}, right_sum = {}"
      .format(*result)
     )
print("left_err = {:.12e}, middle_err = {:.12e}, right_err = {:.12e}"
      .format(*[abs(x - 1) for x in result])
     )

The middle sum gives a better approximation.
Let's analyze the error dependence on $N$.

In [ ]:
N = range(1, 14)
acc = [
    [abs(1 - riemann_sum(np.sin, np.linspace(0, np.pi/2, 2**i), method=method))
     for method in (-1, 0, 1)]
    for i in N
]
acc = np.array(acc).transpose()

plt.figure()

plt.semilogy(N, acc[0], 'b:.', label="$L_N(f)$")
plt.semilogy(N, acc[2], 'm:+', label="$R_N(f)$")
plt.semilogy(N, acc[1], 'r:x', label="$M_N(f)$")
plt.semilogy(N, [2*2**(-1*x) for x in N], '--k', alpha=.5, lw=1, label="$\\propto 2^{-N}$")
plt.semilogy(N, [2**(-2*x)/20 for x in N], '-.k', alpha=.5, lw=1, label="$\\propto 2^{-2N}$")
plt.legend(fontsize=13)
plt.grid()
plt.xlabel("$N$")
plt.ylabel("|error|")

plt.show()

## Exercises

### Approximate Pi

Approximate

$$
\int_0^1 \frac{4}{1 + x^2} dx
$$

to within $10^{-5}$.

In [ ]:
exact_value = np.pi
target_acc = 10**(-5)

# calculate Riemann sum with an increasing N
# until the target accuracy is reached
current_acc = 1
N = 2
mid_sum = 0
while current_acc > target_acc and N < 200:
    mid_sum = riemann_sum(lambda x: 4/(1 + x*x), np.linspace(0, 1, N), method=0)
    current_acc = abs(exact_value - mid_sum)
    N += 1
    
print(f"approximate value = {mid_sum}, number of partition points = {N}")

### Approximate log(2)

Approximate

$$
\int_1^2 \frac{1}{x} dx
$$

to within $10^{-7}$.

In [ ]:
exact_value = np.log(2)
target_acc = 10**(-7)

current_acc = 1
N = 2
mid_sum = 0
while current_acc > target_acc and N < 1000:
    mid_sum = riemann_sum(lambda x: 1/x, np.linspace(1, 2, N), method=0)
    current_acc = abs(exact_value - mid_sum)
    N += 1
    
print(f"approximate value = {mid_sum}, number of partition points = {N}")

### Error Function

The error function is given by

$$
\mathrm{erf}(x) = \frac{2}{\sqrt\pi} \int_0^x e^{-t^2} dt
$$

Numerically compute the error function for $x \in [0, 3]$ and plot the result.
Compare it with the result calculated from [`scipy.special.erf`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.erf.html).

In [ ]:
from scipy import special # erf is under scipy.special

# 1. divide between 0 and 3 into 30 steps
# 2. in each interval, use Riemann sum to evaluate int_a^b e^(-t^2) dt
nsteps = 30
partition = np.linspace(0, 0.1, 10)
partial_sums = [
    riemann_sum(lambda t: np.exp(-t*t), step*max(partition) + partition)
    for step in range(nsteps)
]

xs = [i * max(partition) for i in range(nsteps + 1)]
approx_erf = np.cumsum(partial_sums) * 2/np.sqrt(np.pi)
approx_erf = np.hstack(([0], approx_erf)) # erf(0) = 0

x_exact = np.linspace(0, 3, 200)
y_exact = special.erf(x_exact)

plt.figure()

plt.plot(x_exact, y_exact, "-k", label="exact")
plt.plot(xs, approx_erf, ".:r", lw=1, label="approx")
plt.legend()

plt.show()

### Double Integral

We can use `riemann_sum` (or other numerical integration methods) to carry out multiple integrals.

Consider a double integral

$$
I = \int_{1}^{2} \int_{0}^{1} y^x dx dy
\approx 1.2292741342880657562
$$

This can be written as

$$
I = \int_{1}^{2} F(y) dy
$$

where

$$
F(y) = \int_{0}^{1} y^x dx
$$

We can approximate $I$ using two nested Riemann sums.

In [ ]:
def inner_sum(y):
    return riemann_sum(lambda x: y**x, np.linspace(0, 1, 200))
    
outer_sum = riemann_sum(inner_sum, np.linspace(1, 2, 200))

print("approximatio = {}, absolute error = {}"
      .format(outer_sum, abs(outer_sum - 1.2292741342880657562)))